# How to handle large numbers of tools

:::info Prerequisites

This guide assumes familiarity with the following concepts:

- [Tools](/docs/concepts#tools)
- [Tool calling](/docs/concepts/#functiontool-calling)
- [Embeddings](/docs/concepts/#embedding-models) and [vector stores]((/docs/concepts/#vector-stores))

:::

Tool calling allows a model to respond to a given prompt by generating output that matches a user-defined schema. Many LLM providers, including [Anthropic](https://www.anthropic.com/), [Cohere](https://cohere.com/), [Google](https://cloud.google.com/vertex-ai), [Mistral](https://mistral.ai/), [OpenAI](https://openai.com/), and others, support variants of a tool calling feature. These features typically allow requests to the LLM to include available tools and their schemas, and for responses to include calls to these tools.

Importantly, the subset of available tools to call is generally at the discretion of the model (although many providers also enable the user to [specify or constrain the choice of tool](/docs/how_to/tool_choice)). As the number of available tools grows, you may want to limit the scope of the LLM's selection, to decrease token consumption and to help manage sources of error in LLM reasoning.

Here we will demonstrate how to dynamically adjust the tools available to a model. Bottom line up front: like [RAG](/docs/tutorials/rag) and similar methods, we prefix the model invocation by retrieving over available tools. Although we demonstrate one implementation that searches over tool descriptions, the details of the tool selection can be customized as needed.

## Dependencies

**Note**: this guide uses [OpenAI](/docs/integrations/platforms/openai/) for embeddings, but any LangChain embeddings should suffice. We will use a simple [LangGraph](https://langchain-ai.github.io/langgraph/) agent for demonstration purposes.

In [ ]:
%%capture --no-stderr
%pip install -U langchain-core langchain-openai langgraph

We first instantiate a chat model that supports [tool calling](/docs/how_to/tool_calling/):

```{=mdx}
<ChatModelTabs
  customVarName="llm"
/>
```

In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

Let's consider a toy example in which we have one tool for each company in the S&P 500 index. Each tool will fetch information, and is parameterized by a single integer representing the year.

We first construct a registry that associates a unique identifier with a schema for each tool. We will represent the tools using JSON schema, which can be bound directly to [chat models supporting tool calling](/docs/integrations/chat/).

In [3]:
import re
import uuid

def create_tool(company: str) -> dict:
    """Create schema for a placeholder tool."""
    formatted_company = re.sub(r"[^\w\s]", "", company).replace(" ", "_")
    return {
        "title": f"{formatted_company}_information",
        "description": f"Information about {company}.",
        "type": "object",
        "properties": {"year": {"title": "year", "type": "integer"}},
        "required": ["year"],
    }

s_and_p_500_companies = [  # Abbreviated list for demonstration purposes
    "3M",
    "A.O. Smith",
    "Abbott",
    "Accenture",
    "Advanced Micro Devices",
    "Yum! Brands",
    "Zebra Technologies",
    "Zimmer Biomet",
    "Zoetis",
]

tool_registry = {
    str(uuid.uuid4()): create_tool(company)
    for company in s_and_p_500_companies
}

Next, we create a [vector store](/docs/how_to/vectorstores) that will store embeddings of the tool descriptions. This will allow a user query to be associated to a tool via semantic search. This is a simple solution, and in general the full scope of [retrieval solutions](/docs/concepts#retrieval) are available for this step.

In [4]:
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore, VectorStore
from langchain_openai import OpenAIEmbeddings


tool_documents = [
    Document(page_content=tool["description"], id=id)
    for id, tool in tool_registry.items()
]

vector_store = InMemoryVectorStore(embedding=OpenAIEmbeddings())
upsert_response = vector_store.upsert(tool_documents)
assert not upsert_response["failed"]

Finally, we construct our [Runnable](/docs/concepts/#langchain-expression-language-lcel) as follows:

- We create a `retrieve_tools` runnable that will return tools that are relevant to a user query;
- We create a `get_chat_model` runnable that will receive the query and tools, bind the tools to a chat model, and run it on the query.

Note that here we leverage the fact that if a [RunnableLambda](/docs/how_to/functions/) returns an instance of Runnable, that instance is called on its input. So `get_chat_model` only needs to construct the chat model for the chat model to be called on the input query.

In [5]:
from operator import itemgetter
from typing import List, Mapping

from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnablePassthrough,
)


async def _retrieve_tools(
    input: dict,
    vector_store: VectorStore,
    tool_registry: Mapping[str, dict],
) -> List[dict]:
    query = input["query"]
    tool_documents = await vector_store.asimilarity_search(query)
    return [tool_registry[document.id] for document in tool_documents]


async def _get_chat_model(input: dict) -> Runnable:
    model = llm.bind_tools(input["tools"])
    return itemgetter("query") | model


retrieve_tools = RunnableLambda(_retrieve_tools).bind(
    vector_store=vector_store,
    tool_registry=tool_registry,
)

get_chat_model = RunnableLambda(_get_chat_model)

chain = RunnablePassthrough.assign(tools=retrieve_tools) | get_chat_model

Invoking the chain, we see that the retriever step is able to recover an appropriate tool, and the LLM is able to translate the user's query into an invocation of the tool:

In [6]:
response = await chain.ainvoke({"query": "Can you give me some information about AMD in 2022?"})
response.tool_calls

[{'name': 'Advanced_Micro_Devices_information',
  'args': {'year': 2022},
  'id': 'call_jgQ4Hgt5Svw0YJ9dFpGYinzO',
  'type': 'tool_call'}]

See [LangSmith trace](https://smith.langchain.com/public/0a298c50-1b88-4914-8007-db4c98a4d3e4/r) for the above run.